In [1]:
!pip install gseapy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
############### run single-sample GSEA (ssGSEA) for cancer patient or organoid data ###############
import gseapy
import gseapy as gp
import scipy.stats as stat
import numpy as np
import time, os
import pandas as pd
from collections import defaultdict
cur_dir = os.getcwd()
os.chdir('/content/drive/MyDrive/4060HW/final/organoid_biomarker_detection/utilities')
exec(compile(open('pathway_utilities.py', "rb").read(), 'pathway_utilities.py', 'exec'), globals())
exec(compile(open('parse_preclinical_model_data.py', "rb").read(), 'parse_preclinical_model_data.py', 'exec'), globals())
exec(compile(open('parse_patient_expression.py', "rb").read(), 'parse_patient_expression.py', 'exec'), globals())
gene2uniprot, uniprot2gene = geneID2uniprot(), uniprot2geneID()
os.chdir(cur_dir)

In [4]:
## INITIALIZE
#======================
# INITIALIZE PARAMETERS
source = 'organoid' # 'organoid', 'TCGA'
cancer_type = 'COAD'
testing_pathway_list = ['REACTOME']

In [5]:
#==================
# IMPORT EXPRESSION
print('importing expression for %s, ' %source, time.ctime())

expDic = {} # { sample ID : { gene in uniprot : exp } }
expDic_geneID = {} # { sample ID : { gene : exp } }
geneList, sampleList = [], []

if source.lower() == 'organoid':
	expDic_geneID, expDic = parse_organoid_transcriptome(cancer_type)

if source.upper() == 'TCGA':
	expDic_geneID, expDic = parse_TCGA_log2_FPKM(cancer_type)

importing expression for organoid,  Mon Dec 20 21:48:16 2021


In [6]:
expDic_geneID

{'P10': {'UBE2Q1': 10.77948,
  'RNF14': 10.64877,
  'RNF17': 2.92829,
  'RNF10': 10.14697,
  'RNF11': 9.57152,
  'RNF13': 10.15992,
  'REM1': 6.10365,
  'REM2': 6.20976,
  'RPEL1': 4.06293,
  'UBE2Q2': 8.5791,
  'CHMP1B': 10.36585,
  'PMM2': 9.38149,
  'PMM1': 6.6814,
  'ASS1': 11.49823,
  'SPX': 5.09256,
  'ZNF709': 6.14828,
  'ZNF708': 6.30938,
  'ZNF879': 5.74496,
  'ZNF878': 6.12071,
  'ATRIP': 7.41205,
  'CAMK1': 7.80589,
  'COPRS': 7.80838,
  'ZNF700': 6.17027,
  'ZNF707': 6.18964,
  'CAMK4': 4.13315,
  'ZNF704': 8.76118,
  'ZC3H10': 7.22052,
  'ZC3H13': 9.62685,
  'RNF115': 10.36307,
  'RNF112': 6.08683,
  'ZC3H14': 9.97393,
  'SPN': 6.1578,
  'RNF111': 8.76919,
  'ZC3H18': 9.6241,
  'DHX8': 9.51264,
  'SP9': 6.4227,
  'LAS2': 8.81357,
  'TCOF1': 8.62818,
  'NSRP1': 8.3861,
  'NUP98': 10.25103,
  'SP2': 8.42456,
  'SP3': 8.75362,
  'NUP93': 10.36157,
  'SP5': 8.51198,
  'SP6': 7.17432,
  'CAMKV': 6.05656,
  'SPPL3': 9.06129,
  'GOLIM4': 9.45325,
  'OPA1': 9.81035,
  'CBX1': 8.09

In [7]:
sampleList = list(expDic.keys())
sampleList.sort()

for sample in expDic_geneID:
	if len(geneList) == 0:
		geneList = list(expDic_geneID[sample].keys())
	else:
		geneList = list(set(geneList).intersection(list(expDic_geneID[sample].keys())))

In [8]:
sampleList

['P10',
 'P11',
 'P14',
 'P16',
 'P17',
 'P18',
 'P19A',
 'P19B',
 'P20',
 'P23',
 'P24A',
 'P24B',
 'P25',
 'P26',
 'P27',
 'P28',
 'P31',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9']

In [9]:
#========================================
# IMPORT PATHWAYS FOR ENRICHMENT ANALYSIS
print('importing pathways, ', time.ctime())
reactome = reactome_genes_uniprot() # { pathway : [ gene list ] }
pathwayDic = {'reactome':reactome} # 

importing pathways,  Mon Dec 20 21:48:19 2021


In [10]:
pathwayDic

{'reactome': defaultdict(list,
             {'REACTOME_3_UTR_MEDIATED_TRANSLATIONAL_REGULATION': ['P42766',
               'P47813',
               'P05198',
               'P41091',
               'P60842',
               'Q14240',
               'P23588',
               'P06730',
               'Q04637',
               'P35544',
               'P40429',
               'Q9Y3U8',
               'P11940',
               'Q9UBQ5',
               'Q9UHL3',
               'P60228',
               'P08865',
               'P62906',
               'Q9UNX3',
               'P39023',
               'Q92901',
               'P36578',
               'P46777',
               'Q02878',
               'P18124',
               'P62424',
               'P62917',
               'P32969',
               'P27635',
               'P62913',
               'P30050',
               'P26373',
               'P61313',
               'P18621',
               'Q07020',
               'Q02543',
               'P

In [11]:
len(pathwayDic['reactome'])

674

In [12]:
## PRINT ssGSEA RESULTS

#===============
# MAKE DIRECTORY
fo_directory = '/content/drive/MyDrive/4060HW/final/organoid_biomarker_detection/python/results' 
dir_list = [cancer_type.upper(), source]
for d in dir_list:
	if os.path.isdir('%s/%s' %(fo_directory, d)) == False:
		os.mkdir('%s/%s' %(fo_directory, d))
	fo_directory = '%s/%s'%(fo_directory, d)

In [13]:
#=====================
# MAKE GSEA INPUT FILE
fiList = os.listdir(fo_directory)

# gene expression
if not 'expression.txt' in fiList:
    fo = open('%s/expression.txt' %(fo_directory), 'w')
    print('\t'.join(['NAME', 'DESCRIPTION']) + '\t' + '\t'.join(sampleList), file=fo)
    for gene in geneList:
        tmp = [gene, 'na']
        for sample in sampleList:
            tmp.append(expDic_geneID[sample][gene])
        print('\t'.join(map(str, tmp)), file=fo)
    fo.close()

In [14]:
pd.read_csv("/content/drive/MyDrive/4060HW/final/organoid_biomarker_detection/python/results/COAD/organoid/expression.txt",sep="\t")

,NAME,DESCRIPTION,P10,P11,P14,P16,P17,P18,P19A,P19B,P20,P23,P24A,P24B,P25,P26,P27,P28,P31,P5,P6,P7,P8,P9
0,PRDX4,na,8.28831,8.43979,7.43698,7.88521,7.71150,8.33795,7.41675,7.77650,8.22684,7.63490,7.66663,7.79357,7.81878,7.98213,7.39570,7.23584,8.26692,6.76700,8.52700,8.04964,8.64941,8.12113
1,KIF14,na,8.81675,9.17991,8.42032,9.27511,8.72921,9.02402,9.28737,9.26667,9.05926,7.70811,9.39023,8.68172,8.80634,9.09823,7.36313,8.55869,8.34240,5.50599,9.19447,8.68172,9.34181,8.02769
2,A2M,na,4.91493,8.82575,7.09829,4.52842,4.62305,5.59516,4.29090,4.43726,4.83360,4.53434,4.52926,4.48442,5.27913,5.91586,5.04807,4.66653,4.58574,8.67648,4.67001,5.05694,4.96230,4.69321
3,RNF181,na,8.06515,8.38064,8.31152,8.10314,8.18201,8.56549,8.49306,8.52287,8.50722,8.41202,8.14315,8.19591,8.46373,8.19186,8.38016,8.58928,8.67194,8.31490,8.24264,8.25993,8.48888,8.55369
4,GPKOW,na,9.47400,9.72647,9.29301,9.60407,9.18785,9.53386,9.51222,9.65547,9.35914,8.71692,9.27394,9.35464,9.67159,9.02721,9.10879,9.20111,9.63783,8.68461,9.44482,9.54876,9.43042,9.31832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12032,PAK4,na,9.20473,9.38553,9.06889,8.94440,9.41133,8.96510,9.40405,9.12697,9.13980,9.21998,9.35179,9.31431,9.33745,9.27606,8.83225,8.89731,9.14546,9.02402,9.20545,8.89329,9.45998,9.22834
12033,CLPTM1L,na,10.69320,10.36516,10.25222,10.16406,10.34185,10.39165,10.03374,10.26140,10.52210,9.90966,10.52871,10.12399,10.60957,10.24139,10.18929,9.99806,9.88183,9.63067,10.65055,10.14327,10.48577,9.56636
12034,ZNF280C,na,10.45445,9.66891,10.10681,10.09980,11.02753,10.60998,11.00596,9.33141,10.37585,9.75379,10.42265,9.95564,10.00191,10.33989,9.46618,9.70590,10.52432,9.49146,10.68496,10.07515,10.03848,11.10635
12035,BAG5,na,8.44805,8.67546,8.38270,8.61418,9.33398,8.12904,8.49737,8.96516,8.19136,8.60472,8.90343,9.05439,8.70909,8.55973,8.88517,8.61568,8.39246,8.25100,8.57405,8.82392,8.33335,7.98719


In [15]:
#=======
# ssGSEA
for testing_pathway in testing_pathway_list:
    if testing_pathway.lower() in pathwayDic:
        print('running ssGSEA for %s ... , ' %testing_pathway.lower(), time.ctime())
        # gene sets for ssGSEA
        gene_sets = {}
        pw_list = []
        for pw in pathwayDic[testing_pathway.lower()]:
            for uniprot in pathwayDic[testing_pathway.lower()][pw]:
                if uniprot in uniprot2gene:
                    gene = uniprot2gene[uniprot]
                    if gene in geneList:
                        if not pw in gene_sets:
                            gene_sets[pw] = []
                        gene_sets[pw].append(gene)
        pw_list = list(gene_sets.keys())

        fo = open('%s/%s.gmt' %(fo_directory, testing_pathway.lower()), 'w')
        for pw in pw_list:
            print(pw + '\t' + '\t'.join(gene_sets[pw]), file=fo)
        fo.close()


        # ssGSEA
        ss = gp.ssgsea(data='%s/expression.txt'%(fo_directory), outdir='%s/%s_ssgsea_result'%(fo_directory, testing_pathway.lower()),
                       gene_sets='%s/%s.gmt' %(fo_directory, testing_pathway.lower()),
                       sample_norm_method='rank', permutation_num=0, no_plot=True, scale=True, min_size=2)

running ssGSEA for reactome ... ,  Mon Dec 20 21:48:20 2021
